In [11]:
import phrase_entity_extraction as ph

In [12]:
sent = "701 LUPO LANE APT E23 HUNTERS RUN".lower()
max_address = ph.return_max_address3(ph.seq, sent)
if sent.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(sent))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(sent), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(sent.upper(), encoding))

max len 3
A [[0, 2, 2, ['ADDRESS'], '701 lupo'], [4, 6, 2, ['ADDRESS'], 'e23 hunters'], [0, 3, 3, ['ADDRESS'], '701 lupo lane'], [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]
B [[0, 3, 3, ['ADDRESS'], '701 lupo lane'], [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]
C [[4, 7, 3, ['ADDRESS'], 'e23 hunters run'], [0, 3, 3, ['ADDRESS'], '701 lupo lane']]
C [[0, 1, 1, ['SUITE'], '701'], [4, 5, 1, ['SUITE'], 'e23'], [3, 5, 2, ['SUITE'], 'apt e23']]

## ************** NOT THE SAME!!!! *********************##

Max Address:  701 LUPO LANE APT E23 

WORD ARRAY:  ['701', 'lupo', 'lane', 'apt', 'e23', 'hunters', 'run'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['WORDWAY'], ['APT'], ['ADR_HEAD', 'ALNUM'], ['ALPHA'], ['ALPHA']]


701 LUPO LANE APT E23 HUNTERS RUN,"[['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['WORDWAY'], ['APT'], ['ADR_HEAD', 'ALNUM'], ['ALPHA'], ['ALPHA']]"


In [13]:
ph.return_best_fit(ph.seq, sent)

max len 3
A [[0, 2, 2, ['ADDRESS'], '701 lupo'], [4, 6, 2, ['ADDRESS'], 'e23 hunters'], [0, 3, 3, ['ADDRESS'], '701 lupo lane'], [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]
B [[0, 3, 3, ['ADDRESS'], '701 lupo lane'], [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]
C [[4, 7, 3, ['ADDRESS'], 'e23 hunters run'], [0, 3, 3, ['ADDRESS'], '701 lupo lane']]
C [[0, 1, 1, ['SUITE'], '701'], [4, 5, 1, ['SUITE'], 'e23'], [3, 5, 2, ['SUITE'], 'apt e23']]


[[0, 3, 3, ['ADDRESS'], '701 lupo lane'], [3, 5, 2, ['SUITE'], 'apt e23']]

In [14]:
arr_cands = ph.get_markers(ph.seq, sent, ['ADDRESS', 'POBOX', 'SUITE'])
arr_cands

[[0, 1, 1, ['SUITE'], '701'],
 [0, 2, 2, ['ADDRESS'], '701 lupo'],
 [0, 3, 3, ['ADDRESS'], '701 lupo lane'],
 [3, 5, 2, ['SUITE'], 'apt e23'],
 [4, 5, 1, ['SUITE'], 'e23'],
 [4, 6, 2, ['ADDRESS'], 'e23 hunters'],
 [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]

In [15]:
def return_best_fit(seq, sent, book_fit=False):
    items_of_interest = ['POBOX', 'ADDRESS', 'ATTN', 'SUITE','_DIR_']
    markers = ph.get_markers(seq, sent, items_of_interest)
    def get_sorted_entity(_markers, entity):
        entities = [arr for arr in _markers if arr[3][0] == entity]
        entities.sort(key=lambda x: int(x[2]))
        return entities
    
    def entitys_overlap(ent1, ent2):
        if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
            return False
        else:
            return True
    
    def add_next(markers, best_fit, entity):
        suites = get_sorted_entity(markers, entity)
        idx = len(suites) - 1
        while idx >= 0:
            if not any([entitys_overlap(item, suites[idx]) for item in best_fit]):
                best_fit.append(suites[idx])
                break
            else:
                idx -= 1
        return best_fit
    
    def book_best_fit(arr_domain, markers):
        def is_in_dictionary(markers, start, endplus):
            match_starts = [item for item in markers if item[0] == start]
            match_both = [item for item in match_starts if item[1] == endplus]
            return match_both
        last_length = [-1]*len(arr_domain)
        for i in range(len(arr_domain)):
            if is_in_dictionary(markers, 0, i+1):
                last_length[i] = i + 1
            if last_length[i] == -1:
                for j in range(i):
                    if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                        last_length[i] = i - j
                        break
        decompositions = []
        if last_length[-1] != -1:
            idx = len(arr_domain) - 1
            while idx >= 0:
                decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
                idx -= last_length[idx]
            decompositions = decompositions[::-1]
        return decompositions
    
    best_fit = []
    if not book_fit:
        for nugget in items_of_interest:
            best_fit = add_next(markers, best_fit, nugget)
            best_fit.sort(key=lambda x: int(x[0]))
    else:
        best_fit = book_best_fit(ph.w(sent), markers)

    return best_fit

In [16]:
return_best_fit(ph.seq, sent, book_fit=True)

[]

In [17]:
def return_max_address2(seq, sent):
    kinds = ['ADDRESS', 'POBOX', 'SUITE']
    decomposition = ph.decompose_into_dictionary_words(ph.w(sent.lower()), seq, kinds)
    found_tuples = decomposition[3]
    max_address = []
    for kindof, value in found_tuples:
        if kindof and kindof[0] in kinds:
            max_address.append(value)
    return " ".join(max_address).upper()

In [18]:
ph.decompose_into_dictionary_words(ph.w(sent), ph.seq, ['ADDRESS', 'SUITE', 'POBOX'])

[1, 2, 3, -1, 2, 2, 3]


([1, 2, 3, -1, 2, 2, 3],
 [['SUITE'],
  ['ADDRESS'],
  ['ADDRESS'],
  '',
  ['SUITE'],
  ['ADDRESS'],
  ['ADDRESS']],
 [['ADDRESS'], ['SUITE']],
 [(['ADDRESS'], '701 lupo lane'), (['SUITE'], 'apt e23')])

In [19]:
def book_best_fit(arr_domain, markers):
    def is_in_dictionary(markers, start, endplus):
        match_starts = [item for item in markers if item[0] == start]
        match_both = [item for item in match_starts if item[1] == endplus]
        return match_both
    last_length = [-1]*len(arr_domain)
    for i in range(len(arr_domain)):
        if is_in_dictionary(markers, 0, i+1):
            last_length[i] = i + 1
        if last_length[i] == -1:
            for j in range(i):
                if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                    last_length[i] = i - j
                    break
    decompositions = []
    if last_length[-1] != -1:
        idx = len(arr_domain) - 1
        while idx >= 0:
            decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
            idx -= last_length[idx]
        decompositions = decompositions[::-1]
    return decompositions

book_best_fit(ph.w(sent), arr_cands)

[]

In [20]:
filtered_markers = [marker for marker in arr_cands if not (marker[0] == 0 and marker[2] == 1)]
print(filtered_markers)
arr_cands

[[0, 2, 2, ['ADDRESS'], '701 lupo'], [0, 3, 3, ['ADDRESS'], '701 lupo lane'], [3, 5, 2, ['SUITE'], 'apt e23'], [4, 5, 1, ['SUITE'], 'e23'], [4, 6, 2, ['ADDRESS'], 'e23 hunters'], [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]


[[0, 1, 1, ['SUITE'], '701'],
 [0, 2, 2, ['ADDRESS'], '701 lupo'],
 [0, 3, 3, ['ADDRESS'], '701 lupo lane'],
 [3, 5, 2, ['SUITE'], 'apt e23'],
 [4, 5, 1, ['SUITE'], 'e23'],
 [4, 6, 2, ['ADDRESS'], 'e23 hunters'],
 [4, 7, 3, ['ADDRESS'], 'e23 hunters run']]